In [1]:
from routingpy import Valhalla
from pprint import pprint

# Some locations in Berlin
coords = [[13.413706, 52.490202], [13.421838, 52.514105],
          [13.453649, 52.507987], [13.401947, 52.543373]]
client = Valhalla()

route = client.directions(locations=coords, profile='pedestrian')
isochrones = client.isochrones(locations=coords[0], profile='pedestrian', intervals=[600, 1200])
matrix = client.matrix(locations=coords, profile='pedestrian')

pprint((route.geometry, route.duration, route.distance, route.raw))
pprint((isochrones.raw, isochrones[0].geometry, isochrones[0].center, isochrones[0].interval))
pprint((matrix.durations, matrix.distances, matrix.raw))

([(13.413704, 52.490171),
  (13.413507, 52.490157),
  (13.413443, 52.490152),
  (13.413381, 52.490435),
  (13.413227, 52.491182),
  (13.413169, 52.491434),
  (13.413149, 52.491532),
  (13.413195, 52.491606),
  (13.413254, 52.491639),
  (13.41327, 52.49166),
  (13.413322, 52.491731),
  (13.413341, 52.491756),
  (13.413354, 52.491775),
  (13.413373, 52.4918),
  (13.413428, 52.491875),
  (13.413439, 52.49189),
  (13.413407, 52.491908),
  (13.413511, 52.492055),
  (13.413632, 52.492078),
  (13.413725, 52.492199),
  (13.413922, 52.492461),
  (13.413939, 52.492484),
  (13.413955, 52.492505),
  (13.414359, 52.493053),
  (13.414414, 52.493125),
  (13.414435, 52.493153),
  (13.414457, 52.493182),
  (13.414482, 52.493215),
  (13.414923, 52.493799),
  (13.414953, 52.493838),
  (13.414864, 52.493863),
  (13.415019, 52.494063),
  (13.415159, 52.494245),
  (13.41525, 52.494364),
  (13.415408, 52.494571),
  (13.415417, 52.494583),
  (13.415433, 52.494604),
  (13.415451, 52.494626),
  (13.415459, 52.4

In [6]:
from routingpy import Graphhopper, ORS, MapboxOSRM
from shapely.geometry import Polygon

# Define the clients and their profile parameter
apis = (
   (ORS(api_key='ors_key'), 'cycling-regular'),
   (Graphhopper(api_key='gh_key'), 'bike'),
   (MapboxOSRM(api_key='mapbox_key'), 'cycling')
)
# Some locations in Berlin
coords = [[13.413706, 52.490202], [13.421838, 52.514105],
          [13.453649, 52.507987], [13.401947, 52.543373]]

for api in apis:
    client, profile = api
    route = client.directions(locations=coords, profile=profile)
    print("Direction - {}:\n\tDuration: {}\n\tDistance: {}".format(client.__class__.__name__,
                                                                   route.duration,
                                                                   route.distance))
    isochrones = client.isochrones(locations=coords[0], profile=profile, intervals=[600, 1200])
    for iso in isochrones:
        print("Isochrone {} secs - {}:\n\tArea: {} sqm".format(client.__class__.__name__,
                                                               iso.interval,
                                                               Polygon(iso.geometry).area))
    matrix = client.matrix(locations=coords, profile=profile)
    print("Matrix - {}:\n\tDurations: {}\n\tDistances: {}".format(client.__class__.__name__,
                                                                  matrix.durations,
                                                                  matrix.distances))

RouterApiError: 403 ({
    "error": "Access to this API has been disallowed"
})

In [3]:
import requests
import folium

# Function to query Overpass API for tourist attractions
def get_tourist_places(latitude, longitude, radius=1000):
    overpass_url = "http://overpass-api.de/api/interpreter"
    
    # Query to fetch tourist attractions
    overpass_query = f"""
    [out:json];
    (
      node["tourism"="attraction"](around:{radius},{latitude},{longitude});
      way["tourism"="attraction"](around:{radius},{latitude},{longitude});
      relation["tourism"="attraction"](around:{radius},{latitude},{longitude});
    );
    out center;
    """
    
    response = requests.get(overpass_url, params={'data': overpass_query})
    data = response.json()
    
    return data['elements']

# Function to display locations on a map
def plot_tourist_places_on_map(latitude, longitude, places):
    # Create a folium map centered around the specified location
    tourist_map = folium.Map(location=[latitude, longitude], zoom_start=14)
    
    # Add markers for each tourist place
    for place in places:
        if 'lat' in place and 'lon' in place:
            folium.Marker(
                [place['lat'], place['lon']],
                popup=place.get('tags', {}).get('name', 'Unknown Attraction')
            ).add_to(tourist_map)
    
    # Save the map as an HTML file
    tourist_map.save("tourist_places_map.html")
    print("Map saved as tourist_places_map.html")

# Example usage
latitude = 48.8566  # Latitude of Paris
longitude = 2.3522  # Longitude of Paris
radius = 2000  # Search radius in meters

# Fetch tourist places near the given location
tourist_places = get_tourist_places(latitude, longitude, radius)

# Plot the tourist places on a map
plot_tourist_places_on_map(latitude, longitude, tourist_places)


Map saved as tourist_places_map.html


In [4]:
!open tourist_places_map.html

In [9]:
import asyncio
import json
import math
from typing import List, Optional, TypedDict
from urllib.parse import urljoin

import httpx
from loguru import logger as log
from parsel import Selector
from snippet1 import scrape_location_data, client


class Preview(TypedDict):
    url: str
    name: str


def parse_search_page(response: httpx.Response) -> List[Preview]:
    """parse result previews from TripAdvisor search page"""
    log.info(f"parsing search page: {response.url}")
    parsed = []
    # Search results are contain in boxes which can be in two locations.
    # this is location #1:
    selector = Selector(response.text)
    for box in selector.css("span.listItem"):
        title = box.css("div[data-automation=hotel-card-title] a ::text").getall()[1]
        url = box.css("div[data-automation=hotel-card-title] a::attr(href)").get()
        parsed.append(
            {
                "url": urljoin(str(response.url), url),  # turn url absolute
                "name": title,
            }
        )
    if parsed:
        return parsed
    # location #2
    for box in selector.css("div.listing_title>a"):
        parsed.append(
            {
                "url": urljoin(
                    str(response.url), box.xpath("@href").get()
                ),  # turn url absolute
                "name": box.xpath("text()").get("").split(". ")[-1],
            }
        )
    return parsed


async def scrape_search(query: str, max_pages: Optional[int] = None) -> List[Preview]:
    """scrape search results of a search query"""
    # first scrape location data and the first page of results
    log.info(f"{query}: scraping first search results page")
    try:
        location_data = (await scrape_location_data(query, client))[0]  # take first result
    except IndexError:
        log.error(f"could not find location data for query {query}")
        return
    hotel_search_url = "https://www.tripadvisor.com" + location_data["HOTELS_URL"]

    log.info(f"found hotel search url: {hotel_search_url}")
    first_page = await client.get(hotel_search_url)
    assert first_page.status_code == 200, "scraper is being blocked"

    # parse first page
    results = parse_search_page(first_page)
    if not results:
        log.error("query {} found no results", query)
        return []

    # extract pagination metadata to scrape all pages concurrently
    page_size = len(results)
    total_results = first_page.selector.xpath("//span/text()").re(
        "(\d*\,*\d+) properties"
    )[0]
    total_results = int(total_results.replace(",", ""))
    next_page_url = first_page.selector.css(
        'a[aria-label="Next page"]::attr(href)'
    ).get()
    next_page_url = urljoin(hotel_search_url, next_page_url)  # turn url absolute
    total_pages = int(math.ceil(total_results / page_size))
    if max_pages and total_pages > max_pages:
        log.debug(
            f"{query}: only scraping {max_pages} max pages from {total_pages} total"
        )
        total_pages = max_pages

    # scrape remaining pages
    log.info(
        f"{query}: found {total_results=}, {page_size=}. Scraping {total_pages} pagination pages"
    )
    other_page_urls = [
        # note: "oa" stands for "offset anchors"
        next_page_url.replace(f"oa{page_size}", f"oa{page_size * i}")
        for i in range(1, total_pages)
    ]
    # we use assert to ensure that we don't accidentally produce duplicates which means something went wrong
    assert len(set(other_page_urls)) == len(other_page_urls)

    to_scrape = [client.get(url) for url in other_page_urls]
    for response in asyncio.as_completed(to_scrape):
        results.extend(parse_search_page(await response))
    return results

# example use:
if __name__ == "__main__":

    async def run():
        result = await scrape_search("Malta", client)
        print(json.dumps(result, indent=2))

    asyncio.run(run())

<>:73: SyntaxWarning: invalid escape sequence '\d'
<>:73: SyntaxWarning: invalid escape sequence '\d'
/var/folders/_m/654bzpvx5jd51wnsbj75kqg00000gn/T/ipykernel_8694/491539341.py:73: SyntaxWarning: invalid escape sequence '\d'
  "(\d*\,*\d+) properties"
/var/folders/_m/654bzpvx5jd51wnsbj75kqg00000gn/T/ipykernel_8694/491539341.py:73: SyntaxWarning: invalid escape sequence '\d'
  "(\d*\,*\d+) properties"


ModuleNotFoundError: No module named 'snippet1'

In [8]:
import requests
from bs4 import BeautifulSoup

def get_tripadvisor_ratings(place_url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    
    # Send a GET request to the TripAdvisor page
    response = requests.get(place_url, headers=headers)
    
    # Parse the content with BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Extract the rating value (it may differ based on the HTML structure)
    try:
        rating = soup.find('span', {'class': 'r2Cf69'}).text  # TripAdvisor uses this class for ratings
        return rating
    except AttributeError:
        return None

# Example usage
place_url = "https://www.tripadvisor.com/Attraction_Review-g187147-d188107-Reviews-Eiffel_Tower-Paris_Ile_de_France.html"
rating = get_tripadvisor_ratings(place_url)
print(f"Rating: {rating}")


Rating: None


In [7]:
import requests
import overpy

def get_osm_tourist_places(lat, lon, radius=1000):
    overpass_url = "http://overpass-api.de/api/interpreter"
    overpass_query = f"""
    [out:json];
    (
      node["tourism"="attraction"](around:{radius},{lat},{lon});
      way["tourism"="attraction"](around:{radius},{lat},{lon});
      relation["tourism"="attraction"](around:{radius},{lat},{lon});
    );
    out center;
    """
    response = requests.get(overpass_url, params={'data': overpass_query})
    return response.json()['elements']

# Example usage
lat, lon = 48.8566, 2.3522  # Paris coordinates
osm_tourist_places = get_osm_tourist_places(lat, lon)

for place in osm_tourist_places:
    print(place.get('tags', {}).get('name', 'Unnamed'))


Point zéro des Routes de France
Robinier du square René Viviani
Le Petit Bouillon Pharamond
Tours de Notre-Dame
Tour de l'Horloge
Rue des Rosiers
Rue des barres
Rivoli 59
Unnamed
Unnamed
Université Paris 1 Panthéon-Sorbonne - Centre Sorbonne
Tour Saint-Jacques
Collège des Bernardins
Pont Neuf
Pont Neuf
Pont au Change
Hôtel de Lauzun
Centre Georges Pompidou
Centre Wallonie-Bruxelles
Unnamed
Cathédrale Notre-Dame de Paris
Unnamed
Unnamed
Maison dite « de Jacques Cœur »
Maison à l'enseigne du Faucheur
Maison à l'enseigne du Mouton
Unnamed
Hôtel de Ville
Palais de Justice de Paris
Maison de Nicolas Flamel
Square Louis-XIII
La Samaritaine
Palais du Louvre
Hôtel de Soubise
Mémorial des Martyrs de la Déportation


In [16]:
import requests
from bs4 import BeautifulSoup

def get_tripadvisor_ratings(place_name):
    # Create a formatted URL for TripAdvisor search
    search_url = f"https://www.tripadvisor.com/Search?q={place_name.replace(' ', '+')}"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    
    # Send a GET request
    response = requests.get(search_url, headers=headers)
    
    # Parse the content with BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')
    
    try:
        # Try to extract the rating from the page
        rating = soup.find('span', {'class': 'r2Cf69'}).text  # This class may vary
        return rating
    except AttributeError:
        return None

# Alternatively, search Yelp for ratings (if TripAdvisor fails)
def get_yelp_ratings(place_name, location="Paris"):
    search_url = f"https://www.yelp.com/search?find_desc={place_name.replace(' ', '+')}&find_loc={location}"
    print(search_url)
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    response = requests.get(search_url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    print(soup)
    
    try:
        # Look for the tag with the correct 'data-traffic-crawl-id' attribute
        rating_tag = soup.find('div', {'data-traffic-crawl-id': 'SearchResultBizRating'})
        print(rating_tag)
        if rating_tag:
            rating = rating_tag.get_text(strip=True)
            return rating
    except AttributeError:
        return None

def get_rating(place_name):
    # First, try TripAdvisor
    rating = get_tripadvisor_ratings(place_name)
    if not rating:
        # If no rating on TripAdvisor, fallback to Yelp
        rating = get_yelp_ratings(place_name)
    return rating

# Example usage
place = "Le Petit Bouillon Pharamond"
rating = get_rating(place)
print(f"Place: {place}, Rating: {rating}")


https://www.yelp.com/search?find_desc=Le+Petit+Bouillon+Pharamond&find_loc=Paris
<html><head><title>yelp.com</title><style>#cmsg{animation: A 1.5s;}@keyframes A{0%{opacity:0;}99%{opacity:0;}100%{opacity:1;}}</style></head><body style="margin:0"><p id="cmsg">Please enable JS and disable any ad blocker</p><script data-cfasync="false">var dd={'rt':'c','cid':'AHrlqAAAAAMAZGDNdT2G2toAXG7Pxw==','hsh':'3BD2468BAE4D73BEA0B5DE8314D745','t':'fe','s':46977,'e':'f06dad4c3a634d016f17e0047d861ca0986d28e493860c470b9c5f2200db839b','host':'geo.captcha-delivery.com','cookie':'ojT~PBt7d0Tb15AwKXMEk977qYmQlvoTm4UA9FEZhsMv4AE2tAdi0oH7NYuQ~ipLNAZc2CrB6eo1kT~uj1Rp4Pc1M5Bbtjkp3cgQPRiAtliVRL~xvgPeFiSpYC7b2ub9'}</script><script data-cfasync="false" src="https://ct.captcha-delivery.com/c.js"></script></body></html>
None
Place: Le Petit Bouillon Pharamond, Rating: None


In [7]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
from webdriver_manager.firefox import GeckoDriverManager
import time

def get_yelp_ratings_with_firefox(place_name, location="Paris"):
    search_url = f"https://www.yelp.com/search?find_desc={place_name.replace(' ', '+')}&find_loc={location}"

    # Set up Firefox WebDriver options
    options = Options()
    options.headless = True  # Run in headless mode (no GUI)

    # Start a Selenium WebDriver session with Firefox
    driver = webdriver.Firefox(service=Service(GeckoDriverManager().install()), options=options)
    driver.get(search_url)

    # Wait for the page to load
    time.sleep(30)  # Adjust time as needed for the page to load

    # Try to find the rating
    try:
        rating_tag = driver.find_element(By.CSS_SELECTOR, 'div[data-traffic-crawl-id="SearchResultBizRating"]')
        rating = rating_tag.text.strip()
        return rating
    except Exception as e:
        print(f"Error extracting rating: {e}")
        return None
    finally:
        driver.quit()

# Example usage
place = "Le Petit Bouillon Pharamond"
rating = get_yelp_ratings_with_firefox(place)
print(f"Place: {place}, Rating: {rating}")


Error extracting rating: Message: Unable to locate element: div[data-traffic-crawl-id="SearchResultBizRating"]; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:511:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16

Place: Le Petit Bouillon Pharamond, Rating: None


In [9]:
import overpy

# Example of combining OSM data with ratings
def combine_osm_with_ratings(lat, lon):
    osm_tourist_places = get_osm_tourist_places(lat, lon)
    
    # Placeholder for final results
    tourist_data = []
    
    for place in osm_tourist_places:
        name = place.get('tags', {}).get('name', 'Unnamed')
        # Scrape rating from TripAdvisor (using the place name to construct the URL)
        tripadvisor_url = f"https://www.tripadvisor.com/Search?q={name.replace(' ', '+')}"
        rating = get_tripadvisor_ratings(tripadvisor_url)
        
        # Add to the list with rating
        tourist_data.append({
            'name': name,
            'rating': rating
        })
    
    return tourist_data

# Example usage
lat, lon = 48.8566, 2.3522  # Paris coordinates
tourist_places_with_ratings = combine_osm_with_ratings(lat, lon)

for place in tourist_places_with_ratings:
    print(f"Place: {place['name']}, Rating: {place['rating']}")


Place: Point zéro des Routes de France, Rating: None
Place: Robinier du square René Viviani, Rating: None
Place: Le Petit Bouillon Pharamond, Rating: None
Place: Tours de Notre-Dame, Rating: None
Place: Tour de l'Horloge, Rating: None
Place: Rue des Rosiers, Rating: None
Place: Rue des barres, Rating: None
Place: Rivoli 59, Rating: None
Place: Unnamed, Rating: None
Place: Unnamed, Rating: None
Place: Université Paris 1 Panthéon-Sorbonne - Centre Sorbonne, Rating: None
Place: Tour Saint-Jacques, Rating: None
Place: Collège des Bernardins, Rating: None
Place: Pont Neuf, Rating: None
Place: Pont Neuf, Rating: None
Place: Pont au Change, Rating: None
Place: Hôtel de Lauzun, Rating: None
Place: Centre Georges Pompidou, Rating: None
Place: Centre Wallonie-Bruxelles, Rating: None
Place: Unnamed, Rating: None
Place: Cathédrale Notre-Dame de Paris, Rating: None
Place: Unnamed, Rating: None
Place: Unnamed, Rating: None
Place: Maison dite « de Jacques Cœur », Rating: None
Place: Maison à l'enseig

In [24]:
async def scrape_location_data(query: str, client: httpx.AsyncClient) -> List[LocationData]:
    log.info(f"scraping location data: {query}")
    
    payload = [
        {
            "variables": {
                "request": {
                    "query": query,
                    "limit": 10,
                    "scope": "WORLDWIDE",
                    "locale": "en-US",
                    "scopeGeoId": 1,
                    "searchCenter": None,
                    "types": ["LOCATION"],
                    "locationTypes": [
                        "GEO", "AIRPORT", "ACCOMMODATION", "ATTRACTION", "EATERY",
                    ],
                    "enabledFeatures": ["articles"],
                    "includeRecent": True,
                }
            },
            "query": "84b17ed122fbdbd4",
            "extensions": {"preRegisteredQueryId": "84b17ed122fbdbd4"},
        }
    ]

    random_request_id = "".join(
        random.choice(string.ascii_lowercase + string.digits) for i in range(180)
    )
    
    headers = {
        "X-Requested-By": random_request_id,
        "Referer": "https://www.tripadvisor.com/Hotels",
        "Origin": "https://www.tripadvisor.com",
    }
    
    result = await client.post(
        url="https://www.tripadvisor.com/data/graphql/ids",
        json=payload,
        headers=headers,
    )
    
    # Try to log the raw data to inspect the structure
    print(result)
    data = json.loads(result.content)
    log.info(f"Raw data received: {json.dumps(data, indent=2)}")
    
    try:
        results = data[0]["data"]["Typeahead_autocomplete"]["results"]
        # Check if the details field exists in any result
        if not results:
            log.warning("No results found in the response")
            return []
        
        # Attempt to extract the 'details' field
        results = [r.get("details") for r in results if "details" in r]
        if not results:
            log.warning("No 'details' found in the results")
        
        log.info(f"Found {len(results)} results with 'details' key")
        return results

    except KeyError as e:
        log.error(f"KeyError: {e}. Response data might have a different structure.")
        return []


In [15]:
!pip3 install "httpx[http2]"

In [33]:
import json
import re
import httpx
import requests

BASE_HEADERS = {
    # "User-Agent": "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.54 Safari/537.36"
}

# hotel_url = "https://www.tripadvisor.com/Hotel_Review-g190327-d264936-Reviews-or10-1926_Hotel_Spa-Sliema_Island_of_Malta.html"
# response = requests.get(hotel_url, headers=BASE_HEADERS)
print(response)
hidden_data = re.findall(r"pageManifest:({.+?})};", response.text, re.DOTALL)[0]
hidden_data = json.loads(hidden_data)['urqlCache']['results']
review_data = json.loads(next(v["data"] for k, v in hidden_data.items() if '"reviewListPage"' in v["data"]))
for review in review_data['locations'][0]['reviewListPage']['reviews']:
    print(review)

<Response [200]>


IndexError: list index out of range

In [35]:
import requests

cookies = {
    'TADCID': 'Yl1Dwf63VeNM4dmbABQCrj-Ib21-TgWwDB4AzTFpg4DwIBbfuvYNQ6SfuVZDJRxbnSNKroTlPCWqZihuGLvIjwSCqDjHcAx1LgU',
    'TASameSite': '1',
    'TAUnique': '%1%enc%3AEbbSgXvMWO69mN4wpYrLx7zALTx10Y8AZfDAtHo0f3PugT4RscsDMS3%2F5lmbpTjVNox8JbUSTxk%3D',
    '__vt': 'SjQ5UyA9vjww6Mq9ABQCjdMFtf3dS_auw5cMBDN7STDNwLvRcpcOGSx4uCMuDBQbEBzMysZRdNHVwFZOobZZw8q6VN4VCy9k2hcMX5EI6yDcPlOjONAQJ6a0MY-hsonKalMMT5rhSM7aa66XNVvRW3JWzg',
    'TASSK': 'enc%3AAKe61d%2FfLureqAzj9R%2BjekiazERzXBBP2w5ZaT%2BabKvtFVXIgY2MlANi7GWMZwsz4OR%2Fl%2Fdd%2FzrRk2pawwBsdMzh4ppZlgb2%2Ft15Z%2BLrdnJhOrgkCCKOgTe%2BWAmW3Fw1UQ%3D%3D',
    'TASession': 'V2ID.BAD1D38F38764000A58CE622F83FBEFA*SQ.3*LS.Hotel_Review*HS.recommended*ES.popularity*DS.5*SAS.popularity*FPS.oldFirst*FA.1*DF.0*TRA.true*LD.264936*EAU._',
    'PAC': 'APMG0wUtQhI4dJ3yO06c26dTVqXH-fMd1AzpDHFMfJ_0Gh6QE_WEOq02tGE9msecB2FuDYzshkpH72zs_yb-lhinP1KlcSM9fbJoZMM--z1iWf6TMY8lIqmM_m66FmTA0-0tZAJoOPG7dwFpXDOvd3yXtxqoxtjxxPhyE3Kt3EJx44q8kLpQw-bk-cYj_0NMUoA57YzGW7qNbcur3fx-876-C-kQQ7rDcC1a5tiv1FJqnKNpZrUJe6wripNzaO9qXQ%3D%3D',
    'SRT': 'TART_SYNC',
    'ServerPool': 'B',
    'PMC': 'V2*MS.81*MD.20241015*LD.20241015',
    'TART': '%1%enc%3AOcMXAMd0FnxUroKsnMJoFb6R6rx2xGCXrDF%2BgRob97a8yKIi5nzfM%2BXFgyiROOHdutvME9jnIpc%3D',
    'TATravelInfo': 'V2*A.2*MG.-1*HP.2*FL.3*RS.1',
    'TAUD': 'LA-1729018759474-1*RDD-1-2024_10_16*LG-1-2.1.F.*LD-2-.....',
    'TASID': 'BAD1D38F38764000A58CE622F83FBEFA',
    'datadome': 'DHfdh6y97HrxRmmLTF3Tb5ZD33YoxHgszAABID8n7BvwlGQp7l8GZ51HYvPUEH0ocWIgHK0uBQu~xvlmyIfA_RMOa9F6BftdV39KM2ohCy5HNH1kZZCrgaXvo9NlMKPS',
    'OptanonConsent': 'isGpcEnabled=1&datestamp=Tue+Oct+15+2024+21%3A00%3A08+GMT%2B0200+(Central+European+Summer+Time)&version=202405.2.0&browserGpcFlag=1&isIABGlobal=false&hosts=&consentId=4a903846-0493-499a-ab1f-2c358f049f11&interactionCount=1&isAnonUser=1&landingPath=https%3A%2F%2Fwww.tripadvisor.com%2FHotel_Review-g190327-d264936-Reviews-or10-1926_Le_Soleil_Hotel_Spa-Sliema_Island_of_Malta.html&groups=C0001%3A1%2CC0002%3A1%2CC0003%3A1%2CC0004%3A0',
    'TATrkConsent': 'eyJvdXQiOiJBRFYsU09DSUFMX01FRElBIiwiaW4iOiJBTkEsRlVOQ1RJT05BTCJ9',
}

headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:131.0) Gecko/20100101 Firefox/131.0',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/png,image/svg+xml,*/*;q=0.8',
    'Accept-Language': 'en-US,en;q=0.5',
    # 'Accept-Encoding': 'gzip, deflate, br, zstd',
    'DNT': '1',
    'Sec-GPC': '1',
    'Connection': 'keep-alive',
    # 'Cookie': 'TADCID=Yl1Dwf63VeNM4dmbABQCrj-Ib21-TgWwDB4AzTFpg4DwIBbfuvYNQ6SfuVZDJRxbnSNKroTlPCWqZihuGLvIjwSCqDjHcAx1LgU; TASameSite=1; TAUnique=%1%enc%3AEbbSgXvMWO69mN4wpYrLx7zALTx10Y8AZfDAtHo0f3PugT4RscsDMS3%2F5lmbpTjVNox8JbUSTxk%3D; __vt=SjQ5UyA9vjww6Mq9ABQCjdMFtf3dS_auw5cMBDN7STDNwLvRcpcOGSx4uCMuDBQbEBzMysZRdNHVwFZOobZZw8q6VN4VCy9k2hcMX5EI6yDcPlOjONAQJ6a0MY-hsonKalMMT5rhSM7aa66XNVvRW3JWzg; TASSK=enc%3AAKe61d%2FfLureqAzj9R%2BjekiazERzXBBP2w5ZaT%2BabKvtFVXIgY2MlANi7GWMZwsz4OR%2Fl%2Fdd%2FzrRk2pawwBsdMzh4ppZlgb2%2Ft15Z%2BLrdnJhOrgkCCKOgTe%2BWAmW3Fw1UQ%3D%3D; TASession=V2ID.BAD1D38F38764000A58CE622F83FBEFA*SQ.3*LS.Hotel_Review*HS.recommended*ES.popularity*DS.5*SAS.popularity*FPS.oldFirst*FA.1*DF.0*TRA.true*LD.264936*EAU._; PAC=APMG0wUtQhI4dJ3yO06c26dTVqXH-fMd1AzpDHFMfJ_0Gh6QE_WEOq02tGE9msecB2FuDYzshkpH72zs_yb-lhinP1KlcSM9fbJoZMM--z1iWf6TMY8lIqmM_m66FmTA0-0tZAJoOPG7dwFpXDOvd3yXtxqoxtjxxPhyE3Kt3EJx44q8kLpQw-bk-cYj_0NMUoA57YzGW7qNbcur3fx-876-C-kQQ7rDcC1a5tiv1FJqnKNpZrUJe6wripNzaO9qXQ%3D%3D; SRT=TART_SYNC; ServerPool=B; PMC=V2*MS.81*MD.20241015*LD.20241015; TART=%1%enc%3AOcMXAMd0FnxUroKsnMJoFb6R6rx2xGCXrDF%2BgRob97a8yKIi5nzfM%2BXFgyiROOHdutvME9jnIpc%3D; TATravelInfo=V2*A.2*MG.-1*HP.2*FL.3*RS.1; TAUD=LA-1729018759474-1*RDD-1-2024_10_16*LG-1-2.1.F.*LD-2-.....; TASID=BAD1D38F38764000A58CE622F83FBEFA; datadome=DHfdh6y97HrxRmmLTF3Tb5ZD33YoxHgszAABID8n7BvwlGQp7l8GZ51HYvPUEH0ocWIgHK0uBQu~xvlmyIfA_RMOa9F6BftdV39KM2ohCy5HNH1kZZCrgaXvo9NlMKPS; OptanonConsent=isGpcEnabled=1&datestamp=Tue+Oct+15+2024+21%3A00%3A08+GMT%2B0200+(Central+European+Summer+Time)&version=202405.2.0&browserGpcFlag=1&isIABGlobal=false&hosts=&consentId=4a903846-0493-499a-ab1f-2c358f049f11&interactionCount=1&isAnonUser=1&landingPath=https%3A%2F%2Fwww.tripadvisor.com%2FHotel_Review-g190327-d264936-Reviews-or10-1926_Le_Soleil_Hotel_Spa-Sliema_Island_of_Malta.html&groups=C0001%3A1%2CC0002%3A1%2CC0003%3A1%2CC0004%3A0; TATrkConsent=eyJvdXQiOiJBRFYsU09DSUFMX01FRElBIiwiaW4iOiJBTkEsRlVOQ1RJT05BTCJ9',
    'Upgrade-Insecure-Requests': '1',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'cross-site',
    'Priority': 'u=0, i',
    # Requests doesn't support trailers
    # 'TE': 'trailers',
}

response = requests.get(
    # 'https://www.tripadvisor.com/Hotel_Review-g190327-d264936-Reviews-or10-1926_Le_Soleil_Hotel_Spa-Sliema_Island_of_Malta.html',
    'https://www.tripadvisor.com/Tourism-g187323-Berlin-Vacations.html',
    cookies=cookies,
    headers=headers,
)

In [38]:
response.text

'<!DOCTYPE html><html lang="en-US"><head><link rel="icon" id="favicon" href="https://static.tacdn.com/favicon.ico?v2" type="image/x-icon"/><link rel="mask-icon" sizes="any" href="https://static.tacdn.com/img2/brand_refresh/application_icons/mask-icon.svg" color="#000000"/><meta name="theme-color" content="#34e0a1"/><meta name="format-detection" content="telephone=no"/><meta property="al:ios:app_name" content="TripAdvisor"/><meta property="al:ios:app_store_id" content="284876795"/><meta property="twitter:app:id:ipad" name="twitter:app:id:ipad" content="284876795"/><meta property="twitter:app:id:iphone" name="twitter:app:id:iphone" content="284876795"/><meta property="al:ios:url" content="tripadvisor://www.tripadvisor.com/Tourism-g187323-Berlin-Vacations.html?m=33762"/><meta property="twitter:app:url:ipad" name="twitter:app:url:ipad" content="tripadvisor://www.tripadvisor.com/Tourism-g187323-Berlin-Vacations.html?m=33762"/><meta property="twitter:app:url:iphone" name="twitter:app:url:iph